In [32]:
import pandas as pd
import numpy as np
from region_dpts_France import REGIONS, DEPARTMENTS

## Insertion de la table Région et Département et Ville

Les régions et les départements sont chargés à partir des dictionnaires d'un [module python](./region_dpts_France.py).

In [24]:
#Régions
df_regions = pd.DataFrame(REGIONS.keys())
df_regions = df_regions.rename(columns={0: 'lib_region'})
df_regions.index += 1
df_regions

,lib_region
1,Auvergne-Rhône-Alpes
2,Bourgogne-Franche-Comté
3,Bretagne
4,Centre-Val de Loire
5,Corse
6,Grand Est
7,Guadeloupe
8,Guyane
9,Hauts-de-France
10,Île-de-France


In [35]:
#Départements
df_departement = pd.DataFrame(DEPARTMENTS.items(), columns=['id_dep', 'lib_dep'])
region_for_dep_array = []
for i, row in df_departement.iterrows():
    region = [index+1 for index, [reg, dep_list] in enumerate(REGIONS.items()) if row['id_dep'] in dep_list]
    region_for_dep_array.append(region[0] if len(region) else None)
df_departement['id_region'] = region_for_dep_array
df_departement

,id_dep,lib_dep,id_region
0,01,Ain,1.0
1,02,Aisne,9.0
2,03,Allier,1.0
3,04,Alpes-de-Haute-Provence,18.0
4,05,Hautes-Alpes,18.0
...,...,...,...
99,974,La Réunion,11.0
100,976,Mayotte,13.0
101,978,Saint-Martin,NaN
102,987,Polynésie française,NaN


Les villes sont retrouvées à partir d'un fichier excel listant les IRIS (Ilots Regroupés pour l'Information Statistique) des villes de frances.
Il est donc nécessaire de faire quelques opérations afin de retrouver  notre tableau ville.

Ce fichier sera aussi réutilisé pour calculer les revenus par ville.

<span style="color: red">Attention fichier lourd</span>

In [4]:
#Ville
df_IRIS = pd.read_excel('data/base-ic-evol-struct-pop-2018.xlsx', sheet_name="IRIS", skiprows=5, usecols='A,C,E,F,CE') #CF = Pop Hors ménage (Sans domicile)

In [5]:
df_IRIS_ville = df_IRIS.groupby(['COM', 'DEP', 'LIBCOM']).sum().reset_index().rename(columns={'COM': 'id_ville', 'DEP': 'id_dep','P18_PMEN': 'nb_menages', 'LIBCOM': 'lib_ville'})
df_IRIS_ville['nb_menages'] = df_IRIS_ville['nb_menages'].astype('int64')
df_IRIS_ville

,id_ville,id_dep,lib_ville,nb_menages
0,01001,01,L'Abergement-Clémenciat,771
1,01002,01,L'Abergement-de-Varey,253
2,01004,01,Ambérieu-en-Bugey,13697
3,01005,01,Ambérieux-en-Dombes,1720
4,01006,01,Ambléon,112
...,...,...,...,...
34988,97420,974,Sainte-Suzanne,23614
34989,97421,974,Salazie,7221
34990,97422,974,Le Tampon,78973
34991,97423,974,Les Trois-Bassins,7070


## Insertion de la table Revenus

Pour la tables des revenus disponibles, on rencontre quelques difficultés. Premièrement, les données récupérées ne sont pas par ville mais par IRIS (Ilots Regroupés pour l'Information Statistique), ce qui nous oblige à faire quelques étapes intermédiaires.

Le nombre de ménage par IRIS se situe dans [le fichier](./data/base-ic-evol-struct-pop-2018.xlsx) des villes précédements chargés. On effectue donc tout d'abord **une jointure** de ces deux bases de données en fonction du numéro IRIS.

Deuxièmement, il faut faire attention aux données, certains IRIS ne sont pas répertoriés dans les revenues disponibles, donc on effectuere **un inner-join**.

<span style="color: rgb(150,50,50)">Les données des revenus concernant une ville ne sont donc pas complètes, on moyennera donc la ville avec les IRIS disponibles. On attribura un coefficient du nombre de ménages de l'IRIS sur le nombre de ménages totale des IRIS référenciés.</span>

Commençont par le chargement du fichier des revenus et la jointure.

In [6]:
df_IRIS_disp = pd.read_excel('data/BASE_TD_FILO_DISP_IRIS_2018.xlsx', sheet_name="IRIS_DISP", skiprows=5, usecols='A,E,J,G,Q,U')
df_IRIS_indexed = df_IRIS.set_index('IRIS')
df_IRIS_disp_indexed = df_IRIS_disp.set_index('IRIS')
df_IRIS_merged_disp = pd.concat([df_IRIS_indexed, df_IRIS_disp_indexed], axis=1, join='inner')
df_IRIS_merged_disp = df_IRIS_merged_disp.rename(columns={'DEP': 'id_dep', 'COM': 'id_ville', 'LIBCOM': 'lib_ville', 'P18_PMEN': 'nb_menages', 'DISP_TP6018' :'taux_pauv', 'DISP_D118': 'd1', 'DISP_MED18': 'median', 'DISP_D918': 'd9', 'DISP_PACT18': 'taux_activite'})
df_IRIS_merged_disp

,id_dep,id_ville,lib_ville,nb_menages,taux_pauv,median,d1,d9,taux_activite
IRIS,,,,,,,,,
010040101,01,01004,Ambérieu-en-Bugey,1552.119901,20.1,19300.0,10340.0,31610.0,71.3
010040102,01,01004,Ambérieu-en-Bugey,3588.320330,25.7,17530.0,9970.0,29840.0,69.5
010040201,01,01004,Ambérieu-en-Bugey,4092.419670,19.1,19580.0,11000.0,33210.0,70.9
010040202,01,01004,Ambérieu-en-Bugey,4464.688022,8.9,24530.0,13600.0,40790.0,71.4
010330102,01,01033,Valserhône,2958.728809,18.6,19960.0,10230.0,39080.0,74.4
...,...,...,...,...,...,...,...,...,...
974221001,974,97422,Le Tampon,3196.457258,44.1,14100.0,7160.0,32410.0,74.1
974221101,974,97422,Le Tampon,2372.638529,43.3,14390.0,7900.0,31470.0,73.2
974221201,974,97422,Le Tampon,6924.584727,48.0,13410.0,7310.0,29090.0,66.9


Ensuite on va vouloir connaitre le nombre de ménages total dont on connait les revenus pour chaque ville.

<span style="color: red">Il ne s'agit pas du nombres de ménages total par ville précédément calculé.</span>

In [7]:
df_IRIS_ville_disp = df_IRIS_merged_disp.groupby(['id_dep', 'id_ville', 'lib_ville']).sum().reset_index()[['id_ville', 'nb_menages']]
df_IRIS_ville_disp

,id_ville,nb_menages
0,01004,13697.547924
1,01033,16190.510062
2,01053,38929.004207
3,01173,12839.994636
4,01283,21998.620379
...,...,...
992,97415,102367.367153
993,97416,83187.802676
994,97418,32925.697420
995,97420,22871.955529


Ce nombre total de ménages nous permet d'attribuer un coefficient à chaque IRIS en fonction de sa proportions par rapport aux nombre de ménages prise en compte de la ville.

In [8]:
coeff = []
for i, row in df_IRIS_merged_disp.iterrows():
    total_menages = df_IRIS_ville_disp.loc[df_IRIS_ville_disp['id_ville'] == row['id_ville']]['nb_menages'].values[0]
    coeff.append(row['nb_menages']/total_menages)

df_IRIS_merged_disp['coeff'] = coeff
df_IRIS_merged_disp[['id_ville', 'nb_menages', 'coeff']]

,id_ville,nb_menages,coeff
IRIS,,,
010040101,01004,1552.119901,0.113314
010040102,01004,3588.320330,0.261968
010040201,01004,4092.419670,0.298770
010040202,01004,4464.688022,0.325948
010330102,01033,2958.728809,0.182745
...,...,...,...
974221001,97422,3196.457258,0.040513
974221101,97422,2372.638529,0.030072
974221201,97422,6924.584727,0.087764


Si on additionne les coefficients des IRIS pour chaque ville on a donc bien 1.

In [9]:
df_IRIS_merged_disp.groupby(['id_dep', 'id_ville', 'lib_ville']).sum()[['coeff']]

coeff
id_dep id_ville lib_ville               
01     01004    Ambérieu-en-Bugey    1.0
       01033    Valserhône           1.0
       01053    Bourg-en-Bresse      1.0
       01173    Gex                  1.0
       01283    Oyonnax              1.0
...                                  ...
974    97415    Saint-Paul           1.0
       97416    Saint-Pierre         1.0
       97418    Sainte-Marie         1.0
       97420    Sainte-Suzanne       1.0
       97422    Le Tampon            1.0

[997 rows x 1 columns]

Il nous faut maintenant recalculer chaque variables des IRIS (taux de pauvreté, 1er décil, ...) en applicant son coefficient.

In [10]:
#Code pas optimal
median_coeff = []
d1_coeff = []
d9_coeff = []
taux_pauv_coeff = []
taux_activite_coeff =[]
for i, row in df_IRIS_merged_disp.iterrows():
    coeff = row['coeff']
    median_coeff.append(row['median']*coeff)
    d1_coeff.append(row['d1']*coeff)
    d9_coeff.append(row['d9']*coeff)
    taux_pauv_coeff.append(row['taux_pauv']*coeff)
    taux_activite_coeff.append(row['taux_activite']*coeff)

df_IRIS_merged_coeff = df_IRIS_merged_disp.reset_index()[['IRIS', 'id_ville']]
df_IRIS_merged_coeff['median'] = median_coeff
df_IRIS_merged_coeff['d1'] = d1_coeff
df_IRIS_merged_coeff['d9'] = d9_coeff
df_IRIS_merged_coeff['taux_pauv'] = taux_pauv_coeff
df_IRIS_merged_coeff['taux_activite'] = taux_activite_coeff
df_IRIS_merged_coeff

,IRIS,id_ville,median,d1,d9,taux_pauv,taux_activite
0,010040101,01004,2186.954502,1171.663707,3581.846208,2.277605,8.079267
1,010040102,01004,4592.300442,2611.821757,7817.127506,6.732580,18.206782
2,010040201,01004,5849.921285,3286.472632,9922.159646,5.706512,21.182810
3,010040202,01004,7995.503852,4432.892474,13295.417942,2.900937,23.272685
4,010330102,01033,3647.582862,1869.477589,7141.660233,3.399050,13.596201
...,...,...,...,...,...,...,...
12386,974221001,97422,571.231161,290.071994,1313.021413,1.786617,3.002002
12387,974221101,97422,432.729247,237.565049,946.350896,1.302097,2.201236
12388,974221201,97422,1176.918494,641.556614,2553.061819,4.212684,5.871428
12389,974221202,97422,598.833682,323.043708,1216.139277,1.941699,2.865295


On additionne tout pour retrouver la table de revenu final.

In [11]:
df_revenu_final = df_IRIS_merged_coeff.groupby(['id_ville']).sum()
df_revenu_final

,median,d1,d9,taux_pauv,taux_activite
id_ville,,,,,
01004,20624.680080,11502.850570,34616.551301,17.617634,70.741544
01033,22472.030765,11523.115878,44501.137824,15.901412,78.155982
01053,19338.766058,10662.079385,34202.131656,20.866870,65.171422
01173,32645.352672,13215.815725,67975.112831,9.671756,99.743312
01283,17845.047600,9945.670268,30642.956689,26.402180,67.445883
...,...,...,...,...,...
97415,18142.652494,8609.337160,39417.616718,32.367365,78.287258
97416,16076.537319,8303.118229,35765.373480,38.259666,74.321268
97418,17915.869743,9013.980175,37825.525791,31.908019,81.279353


## Insertion de la table des crimes et des faits

La table des faits est la somme des faits répertorié par les services de police et par les services de la gendarmeries. Ces informations sont dans deux feuille excel différentes.

Chargement des données du excel

In [12]:
df_faits_gn = pd.read_excel('data/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx', sheet_name="Services GN 2018", skiprows=0, header=None)
df_faits_pn = pd.read_excel('data/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx', sheet_name="Services PN 2018", skiprows=0, header=None)


On retrouve d'abord la table Crime (id_crime / lib_crime)

In [13]:
df_crimes = df_faits_gn.drop(0)
df_crimes.columns = df_crimes.iloc[0]
df_crimes = df_crimes[1:].rename(columns={"Code index" : 'id_crime', 'Libellé index \\ CGD': 'lib_crime'})[['id_crime', 'lib_crime']]
df_crimes

1,id_crime,lib_crime
2,1,Règlements de compte entre malfaireurs
3,2,Homicides pour voler et à l'occasion de vols
4,3,Homicides pour d'autres motifs
5,4,Tentatives d'homicides pour voler et à l'occas...
6,5,Tentatives homicides pour d'autres motifs
...,...,...
104,103,Infractions à l'exercice d'une profession règl...
105,104,Infractions au droit de l'urbanisme et de la c...
106,105,Fraudes fiscales
107,106,Autres délits économiques et financiers


Puis on retrouve la table des faits répertorié par la police

In [14]:
df_faits_pn_clean = df_faits_pn.drop(columns=[1]).set_index([0]).T.rename(columns={"Année 2018 - services de police": "id_dep", np.nan: "Périmètres"}).groupby(['id_dep']).sum().drop(columns=['Périmètres', 'Code index'])

df_dict_list_pn = {"id_dep": [], "id_crime": [], "nb_faits": []}
for id_cirme, fait_dict in df_faits_pn_clean.to_dict().items():
    for id_dep, nb_faits in fait_dict.items():
        if nb_faits == 0: continue
        df_dict_list_pn["id_dep"].append(id_dep)
        df_dict_list_pn["id_crime"].append(id_cirme)
        df_dict_list_pn["nb_faits"].append(nb_faits)
df_faits_pn_final = pd.DataFrame.from_dict(df_dict_list_pn).set_index(['id_dep', 'id_crime'])
df_faits_pn_final

,,nb_faits
id_dep,id_crime,
06,1,1
13,1,34
21,1,1
26,1,1
2A,1,2
...,...,...
973,107,164
974,107,534
976,107,103


Vient ensuite celle de la gendarmerie

In [15]:
df_faits_gn_clean = df_faits_gn.drop(columns=[1]).set_index([0]).T.rename(columns={"Année 2018 - compagnies de gendarmerie": "id_dep"})
df_faits_gn_clean = df_faits_gn_clean.groupby(['id_dep']).sum().drop(columns=['Code index'])

df_dict_list_gn = {"id_dep": [], "id_crime": [], "nb_faits": []}
for id_cirme, fait_dict in df_faits_gn_clean.to_dict().items():
    for id_dep, nb_faits in fait_dict.items():
        if nb_faits == 0: continue
        df_dict_list_gn["id_dep"].append(id_dep)
        df_dict_list_gn["id_crime"].append(id_cirme)
        df_dict_list_gn["nb_faits"].append(nb_faits)
df_faits_gn_final = pd.DataFrame.from_dict(df_dict_list_gn).set_index(['id_dep', 'id_crime'])
df_faits_gn_final

,,nb_faits
id_dep,id_crime,
13,1,2
21,1,1
30,1,1
31,1,2
35,1,1
...,...,...
973,107,661
974,107,636
978,107,83


Ensuite on somme ces deux tables pour retrouver notre table des faits final.

<span style="color: rgb(150,50,50)">Il faut bien préciser les index avant cette opération.</span>

In [16]:
df_faits_final = df_faits_gn_final.add(df_faits_pn_final, fill_value=0).astype('int32')
df_faits_final

nb_faits
id_dep id_crime          
01     3                3
       4                1
       5                8
       6                2
       7             1533
...                   ...
988    103              4
       104             22
       105              1
       106             43
       107            506

[9400 rows x 1 columns]

## Insertion de la table des Catégories Socio-Pro et la table Répartion CSP

On charge le fichier et on récupère tout d'abord la table CSP (id_categorie / lib_categorie).

In [17]:
df_csp_load = pd.read_excel('data/CSP_par_region.xlsx', sheet_name="DEP", skiprows=3, skipfooter=2).drop(96)

In [18]:
df_csp_full = df_csp_load.rename(columns={
    'Unnamed: 0': 'id_dep',
    'Unnamed: 1': 'lib_dep',
    'Part des agriculteurs exploitants (en %)': 'Agriculteurs exploitants',
    'Part des artisans, commerçants, chefs d\'entreprises (en %)': 'Artisans, Commerçants, Chefs d\'entreprises',
    'Part des cadres, professions intellectuelles supérieures (en %)': 'Cadres, Professions intellectuelles supérieures',
    'Part des professions intermédiaires (en %)': 'Professions intermédiaires',
    'Part des employés (en %)': 'Employés',
    'Part des ouvriers (en %)': 'Ouvriers',
    'Part des retraités (en %)': 'Retraités',
    'Part des autres personnes sans activité professionnelle (en %)': 'Sans activité professionnelle'
})

df_csp_lib = pd.DataFrame(df_csp_full.columns)[2:].reset_index()[[0]].reset_index().rename(columns={'index': 'id_categorie', 0: 'lib_categorie'})
df_csp_lib[['id_categorie']] += 1
df_csp_lib

,id_categorie,lib_categorie
0,1,Agriculteurs exploitants
1,2,"Artisans, Commerçants, Chefs d'entreprises"
2,3,"Cadres, Professions intellectuelles supérieures"
3,4,Professions intermédiaires
4,5,Employés
5,6,Ouvriers
6,7,Retraités
7,8,Sans activité professionnelle


On a plus qu'à retrouver les répartitions.


In [19]:
id_categorie_list =[]
id_dep_list = []
taux_list = []
for i, row in df_csp_full.iterrows():
    row_cat = row.drop(['id_dep', 'lib_dep'])
    for cat, taux in row_cat.items():
        id_categorie_list.append(df_csp_lib.loc[df_csp_lib['lib_categorie'] == cat].values[0, 0])
        id_dep_list.append(row.id_dep)
        taux_list.append(taux)

df_csp_reparties = pd.DataFrame.from_dict({'id_dep': id_dep_list, 'id_categorie': id_categorie_list, 'taux': taux_list})
df_csp_reparties

,id_dep,id_categorie,taux
0,01,1,0.6
1,01,2,3.9
2,01,3,9.1
3,01,4,16.2
4,01,5,16.4
...,...,...,...
795,974,4,12.2
796,974,5,20.3
797,974,6,13.1
798,974,7,14.9


## Insertion de la table chômage

Certains des noms des départements diffère un petit peu (accents, ...) de ceux que l'on a dans notre table département. On normalise donc pour comparer.

On a aussi une erreur sur 'Val-de-Marne'.

In [20]:
import unicodedata

def normalize_str(str):
    return unicodedata.normalize('NFKD' ,str).encode('ASCII', 'ignore').decode('utf-8')

In [21]:
df_chomage = pd.read_excel("data/ECRT2021-F12.xlsx", sheet_name="Figure 2b", skiprows=2, skipfooter=3, header=None)
df_chomage = df_chomage.rename(columns={0: 'lib_dep', 1: 'taux'}).replace({'Va-de-Marne': 'Val-de-Marne'})

df_departement_normalized = df_departement.copy()
df_departement_normalized['lib_dep'] = df_departement_normalized['lib_dep'].apply(lambda s: normalize_str(s))

id_chomage_dep_list = []

for i, row in df_chomage.iterrows():
    id_chomage_dep_list.append(df_departement_normalized.loc[df_departement_normalized['lib_dep'] == normalize_str(row['lib_dep'])].values[0,0])

df_chomage['id_dep'] = id_chomage_dep_list
df_chomage = df_chomage[['id_dep', 'taux']]
df_chomage

,id_dep,taux
0,01,6.0
1,02,11.1
2,03,8.6
3,04,9.4
4,05,7.5
...,...,...
95,95,8.5
96,971,17.4
97,972,12.4
98,973,16.1


## Insertion dans la BDD

Maintenant que toutes les dataframes sont présentes, il nous reste à les insérer dans la BDD, ajouter les clés primaire et étrangères.


```
database = 'crime'
user = 'root'
mdp = ''
```

In [50]:
import sqlalchemy

sqlEngine = sqlalchemy.create_engine('mysql+pymysql://root:@127.0.0.1/crimes', pool_recycle=3600)
con = sqlEngine.connect()

if_exists = 'replace'

sql_drops = ['csp_repartition', 'fait', 'revenu','chomage', 'ville', 'departement']

try:
    if if_exists == if_exists:
        for i in sql_drops:
            try:
                con.execute('DROP TABLE `%s`;' % i)
            except Exception as ex:
                pass

    #Région
    df_regions.to_sql("region", con, if_exists=if_exists, index_label='id_region')
    con.execute('ALTER TABLE `region` ADD PRIMARY KEY (`id_region`);')
    #Département
    df_departement.to_sql("departement", con, if_exists=if_exists, index=False, dtype={'id_dep': sqlalchemy.NVARCHAR(length=3), 'id_region': sqlalchemy.BIGINT})
    con.execute('ALTER TABLE `departement` ADD PRIMARY KEY (`id_dep`);')
    con.execute('ALTER TABLE `departement` ADD FOREIGN KEY (`id_region`) REFERENCES region(`id_region`);')
    #Ville
    df_IRIS_ville.to_sql("ville", con, if_exists=if_exists, index=False, dtype={'id_ville': sqlalchemy.NVARCHAR(length=5), 'id_dep': sqlalchemy.NVARCHAR(length=3)})
    con.execute('ALTER TABLE `ville` ADD PRIMARY KEY (`id_ville`);')
    con.execute('ALTER TABLE `ville` ADD FOREIGN KEY (`id_dep`) REFERENCES departement(`id_dep`);')
    #Revenus
    df_revenu_final.to_sql('revenu', con, if_exists=if_exists, dtype={'id_ville': sqlalchemy.NVARCHAR(length=5)})
    con.execute('ALTER TABLE `revenu` ADD PRIMARY KEY (`id_ville`);')
    con.execute('ALTER TABLE `revenu` ADD FOREIGN KEY (`id_ville`) REFERENCES ville(`id_ville`);')
    #Crime
    df_crimes.to_sql('crime', con, if_exists=if_exists, index=False, dtype={'id_dep': sqlalchemy.NVARCHAR(length=3)})
    con.execute('ALTER TABLE `crime` ADD PRIMARY KEY (`id_crime`);')
    #Faits
    df_faits_final.to_sql('fait', con, if_exists=if_exists, dtype={'id_dep': sqlalchemy.NVARCHAR(length=3)})
    con.execute('ALTER TABLE `fait` ADD PRIMARY KEY (`id_dep`, `id_crime`);')
    con.execute('ALTER TABLE `fait` ADD FOREIGN KEY (`id_dep`) REFERENCES departement(`id_dep`);')
    con.execute('ALTER TABLE `fait` ADD FOREIGN KEY (`id_crime`) REFERENCES crime(`id_crime`);')
    #Catégorie Socio-Pro List
    df_csp_lib.to_sql('categorie_sp', con, if_exists=if_exists, index=False)
    con.execute('ALTER TABLE `categorie_sp` ADD PRIMARY KEY (`id_categorie`);')
    #Catégorie Socio-Pro Répartition
    df_csp_reparties.to_sql('csp_repartition', con, if_exists=if_exists, index=False, dtype={'id_dep': sqlalchemy.NVARCHAR(length=3)})
    con.execute('ALTER TABLE `csp_repartition` ADD PRIMARY KEY (`id_dep`, `id_categorie`);')
    con.execute('ALTER TABLE `csp_repartition` ADD FOREIGN KEY (`id_dep`) REFERENCES departement(`id_dep`);')
    con.execute('ALTER TABLE `csp_repartition` ADD FOREIGN KEY (`id_categorie`) REFERENCES categorie_sp(`id_categorie`);')
    #Chômage
    df_chomage.to_sql('chomage', con, if_exists=if_exists, index=False, dtype={'id_dep': sqlalchemy.NVARCHAR(length=3)})
    con.execute('ALTER TABLE `chomage` ADD PRIMARY KEY (`id_dep`);')
    con.execute('ALTER TABLE `chomage` ADD FOREIGN KEY (`id_dep`) REFERENCES departement(`id_dep`);')
except ValueError as vx:
    print(vx)
except Exception as ex:
    print(ex)

con.close()